# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt


Bad key savefig.frameon in file C:\Users\sumit\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\sumit\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\sumit\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probab

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [5]:
# Create our features
X =df.copy()
X.drop("loan_status",axis=1,inplace=True)

# Create our target
y = df['loan_status']

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   homeowner         77536 non-null  object 
 3   borrower_income   77536 non-null  int64  
 4   debt_to_income    77536 non-null  float64
 5   num_of_accounts   77536 non-null  int64  
 6   derogatory_marks  77536 non-null  int64  
 7   total_debt        77536 non-null  int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 4.7+ MB


In [7]:
from sklearn.preprocessing import LabelEncoder
# to transform the homeowner column from object to int64 i.e. into 0,1,2
le = LabelEncoder()
le.fit(X['homeowner'])
X['homeowner']=le.transform(X['homeowner'])



In [8]:
X.homeowner.unique()

array([1, 2, 0])

In [9]:
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,stratify=y)

In [12]:
X_train.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
28991,8400.0,6.706,0,43700,0.313501,3,0,13700
20871,8700.0,6.831,2,44900,0.331849,3,0,14900
71204,9000.0,6.957,1,46100,0.349241,3,0,16100
33350,8300.0,6.653,0,43200,0.305556,2,0,13200
19695,9200.0,7.047,1,46900,0.360341,3,0,16900


In [13]:
X_test.dtypes

loan_size           float64
interest_rate       float64
homeowner             int32
borrower_income       int64
debt_to_income      float64
num_of_accounts       int64
derogatory_marks      int64
total_debt            int64
dtype: object

In [14]:
y_train.describe()

count        58152
unique           2
top       low_risk
freq         56277
Name: loan_status, dtype: object

In [15]:
y_test.shape

(19384,)

In [16]:
y_train.shape

(58152,)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [17]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [18]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler=scaler.fit(X_train)

In [19]:
# Scale the training and testing data
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)


# Simple Logistic Regression

In [20]:
# used X_train_scaled in the logistic regerssion as the data has been scaled 

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9885612665920358

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
print('Simple Logistic Regression confusion matrix:')
print(confusion_matrix(y_test, y_pred))

Simple Logistic Regression confusion matrix:
[[  614    11]
 [   99 18660]]


In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



<p>Comment <p> <li> The recall of 0.98 means all the cases which were required to be classified as high-risk were correctly classified. <li>Further, precision of 0.86 means model classified high-risk 86% correctly.

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [36]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=1)
X_resampled,y_resampled=ros.fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [37]:
Counter(y_train) # to compare against the resampled. The nos of high risk in the resampled has been increased

Counter({'low_risk': 56277, 'high_risk': 1875})

In [38]:
# Train the Logistic Regression model using the resampled data
model_os=LogisticRegression(solver='lbfgs',random_state=1)
model_os.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score,confusion_matrix
y_pred=model_os.predict(X_test)

acc_score=balanced_accuracy_score(y_test,y_pred)
acc_score

0.9965079588464203

In [40]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[  624,     1],
       [  101, 18658]], dtype=int64)

In [41]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_pred,y_test))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.86      1.00      0.92      0.93      0.85       725
   low_risk       0.99      1.00      0.86      1.00      0.93      0.87     18659

avg / total       0.99      0.99      0.87      0.99      0.93      0.87     19384



<p>Comment <p> <li> On using Oversampling accuracy marginally improves. 

### SMOTE Oversampling

In [42]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled,y_resampled=SMOTE(random_state=1,sampling_strategy=1.0).fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [43]:
# Train the Logistic Regression model using the resampled data
model_smote=LogisticRegression(solver='lbfgs',random_state=1)
model_smote.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [44]:
# Calculated the balanced accuracy score
y_pred=model_smote.predict(X_test)
acc_score_smote=balanced_accuracy_score(y_test,y_pred)
acc_score_smote

0.9965079588464203

In [45]:
# Display the confusion matrix
print(confusion_matrix(y_test,y_pred))

[[  624     1]
 [  101 18658]]


In [46]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



<p>Comment: <p> <li> precision for high risk is lower than naive random oversampling

# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [47]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc= ClusterCentroids(random_state=1)
X_resampled,y_resampled =cc.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [48]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [49]:
# Calculate the balanced accuracy score
y_pred = model_cc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9853879204648435

In [50]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  610,    15],
       [   98, 18661]], dtype=int64)

In [51]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.97       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [52]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm=SMOTEENN(random_state=1)
X_resampled,y_resampled=sm.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})

In [53]:
# Train the Logistic Regression model using the resampled data
model_smoteenn=LogisticRegression(solver='lbfgs',random_state=1)
model_smoteenn.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [54]:
# Calculate the balanced accuracy score
y_pred=model_smoteenn.predict(X_test)
acc_score_smoteenn=balanced_accuracy_score(y_test,y_pred)
acc_score_smoteenn

0.9964279972279972

In [55]:
# Display the confusion matrix
print(confusion_matrix(y_test,y_pred))

[[  624     1]
 [  104 18655]]


In [56]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   <p> - All the models have very high balanced accuracy score in the range of 0.98 to 0.99. <p> - Amongst the models,Random over sampler, SMOTE, SMOTEENN have 0.995 ACCURACY SCORE.

2. Which model had the best recall score?

    <p> - SMOTE and SMOTEENN have the perfect score of 1.00. Which means the model is able to predict all the cases of high risk. 

3. Which model had the best geometric mean score?

    <p> - SMOTE and SMOTEENN have the geometric mean score of 1.00.

### *So we can use SMOTE or SMOTEENN models to predict the high risk cases.*
